**Section 5.1: Set sampler primitive options such as dynamical decoupling**

This notebook demonstrates how to set options for the SamplerV2 primitive in Qiskit IBM Runtime, focusing on error mitigation techniques like dynamical decoupling and Pauli twirling, as well as shot control. These options are key for the certification exam, particularly for TASK 5.1.

Note: This notebook assumes you have a valid IBM Quantum account.

Replace token = "your_api_token" with your actual token when running.

Replace instance= "your_istance" with your instance

Options like dynamical decoupling and twirling are hardware-dependent and have no effect on ideal simulators.


In [1]:
# Install the other packages via pip
# !pip install qiskit==2.2.0 qiskit-ibm-runtime==0.41.1 qiskit-aer==0.17.2

In [9]:
import os
from qiskit import QuantumCircuit
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler

# Initialize service
service = QiskitRuntimeService(
    # channel="ibm_cloud",
    # token="your_api_token",
    # instance="your_istance"
)


# Select a real backend
# backend = service.least_busy(operational=True, simulator=False)
backend = service.backends()[0]
print(f"Using backend: {backend.name}")

# Create a simple Bell circuit for demonstration
qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)
qc.measure_all()

# Transpile to ISA circuit
pm = generate_preset_pass_manager(optimization_level=1, backend=backend)
isa_qc = pm.run(qc)

Using backend: ibm_brisbane


**5.1.1 Sampler overview & where options live**

The SamplerV2 primitive samples quantum circuits and returns quasi-probability distributions over measurement outcomes. It is useful for estimating probabilities from circuits with measurements.
Options are set on the sampler.options object.
Defaults can be set there, and overridden per-run
if needed.Key options include:
default_shots: Number of circuit executions (default: 4096).
dynamical_decoupling: For idle error mitigation (hardware-only).
twirling: For Pauli twirling to randomize coherent errors (hardware-only).

These are accessed via sampler.options.<category>.<attribute>.



In [4]:
sampler = Sampler(mode=backend)
print(sampler.options)

SamplerOptions(_VERSION=2, max_execution_time=Unset, environment=EnvironmentOptions(log_level='WARNING', callback=None, job_tags=None, private=False), simulator=SimulatorOptions(noise_model=Unset, seed_simulator=Unset, coupling_map=Unset, basis_gates=Unset), default_shots=Unset, dynamical_decoupling=DynamicalDecouplingOptions(enable=Unset, sequence_type=Unset, extra_slack_distribution=Unset, scheduling_method=Unset, skip_reset_qubits=Unset), execution=SamplerExecutionOptionsV2(init_qubits=Unset, rep_delay=Unset, meas_type=Unset), twirling=TwirlingOptions(enable_gates=Unset, enable_measure=Unset, num_randomizations=Unset, shots_per_randomization=Unset, strategy=Unset), experimental=Unset)


**5.1.2 Setting & overriding options (workflow)**

Typical workflow:

1.   Select backend or open a session.
2.   Instantiate SamplerV2 with the backend/session
3.   Set default options on sampler.options.
4.   Run the primitive, overriding if needed.

In [5]:
# Instantiate Sampler
sampler = Sampler(mode=backend)

# Set defaults
sampler.options.default_shots = 2048
sampler.options.dynamical_decoupling.enable = True

# Run with defaults
job_default = sampler.run([isa_qc])
print(job_default.job_id())

# Override for a specific run
job_override = sampler.run([isa_qc], shots=1024)  # Overrides default_shots
print(job_override.job_id())

d3nvem27su3s73f3c08g
d3nvem83qtks738eu2n0


**5.1.3 Shots & measurement return**

default_shots controls how many times the circuit is executed, affecting statistical variance in the probability distribution.

Higher shots reduce variance but increase runtime/cost.

Results are returned as PrimitiveResult with SamplerPubResult, containing quasi-distributions (dict of bitstrings to probabilities).



In [6]:
# Set shots
sampler.options.default_shots = 4000

# Run
job = sampler.run([isa_qc])
result = job.result()

# Access distribution
dist = result[0].data.meas.get_counts()  # Counts; for probabilities, use quasi_dists[0]
print(dist)

{'11': 1886, '00': 1896, '01': 146, '10': 72}


Note: On ideal simulator, Bell state gives ~50% '00', 50% '11'. On hardware, noise causes small probabilities for '01', '10'. More shots decrease variance in estimates.



**5.1.4 Dynamical decoupling (DD)**

DD mitigates idle qubit errors by inserting pulse sequences (e.g., XY4) during idle periods.

Enable with dynamical_decoupling.enable = True.

Select sequence with sequence_type (e.g., 'XY4').

Only affects hardware; no effect on ideal simulators.

In [7]:
# Enable DD with XY4
sampler.options.dynamical_decoupling.enable = True
sampler.options.dynamical_decoupling.sequence_type = "XY4"
sampler.options.dynamical_decoupling.extra_slack_distribution = "middle"
sampler.options.dynamical_decoupling.scheduling_method = "alap"

# Run on hardware
job_dd = sampler.run([isa_qc])
result_dd = job_dd.result()
dist_dd = result_dd[0].data.meas.get_counts()
print(dist_dd)

{'01': 159, '11': 1861, '00': 1904, '10': 76}


Compare distributions with and without DD: DD should show reduced errors (higher counts at '00' and '11').



**5.1.5 TwirlingPauli**

twirling randomizes coherent errors into stochastic Pauli noise, making it easier to mitigate.

Enable gate twirling with twirling.enable_gates = True.

Can combine with DD.

Backend-dependent; affects hardware more.

It increases circuit variants (randomizations), distributing shots across them, which can average out coherent errors and lead to a more symmetric noise distribution.



In [7]:
# Enable twirling
sampler.options.twirling.enable_gates = True
sampler.options.twirling.enable_measure = False  # Optional; measure twirling
sampler.options.twirling.strategy = "active-accum"
sampler.options.twirling.num_randomizations = "auto"  # Or specific int for control

# Combine with DD
sampler.options.dynamical_decoupling.enable = True

# Run
job_twirl = sampler.run([isa_qc])
result_twirl = job_twirl.result()
dist_twirl = result_twirl[0].data.meas.get_counts()
print(dist_twirl)

{'11': 1870, '00': 1907, '01': 145, '10': 78}


In [13]:
# Enable twirling with estimator
# Create a simple Bell circuit for demonstration
from qiskit.quantum_info import SparsePauliOp
from qiskit_aer.primitives import EstimatorV2

qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)
# Transpile to ISA circuit
pm = generate_preset_pass_manager(optimization_level=1, backend=backend)
cir_isa = pm.run(qc)
obs_isa = SparsePauliOp('Z')
estimator=EstimatorV2()
estimator.options.twirling.enable_gates = True
estimator.options.twirling.enable_measure = False  # Optional; measure twirling
estimator.options.twirling.strategy = "active-accum"
estimator.options.twirling.num_randomizations = "auto"  # Or specific int for control

# Run
job_twirl = estimator.run([cir_isa, obs_isa], precision=0.5)
result_twirl = job_twirl.result()
# dist_twirl = result_twirl[0].data.meas.get_counts()
# print(dist_twirl)

AttributeError: 'Options' object has no attribute 'twirling'

In [20]:
from qiskit import QuantumCircuit
from qiskit.quantum_info import SparsePauliOp
from qiskit.transpiler import generate_preset_pass_manager
from qiskit_ibm_runtime.fake_provider import FakeManilaV2
from qiskit_aer.noise import NoiseModel
from qiskit_aer import AerSimulator
from qiskit_ibm_runtime import EstimatorV2

fake_backend = FakeManilaV2()
noise_model = NoiseModel.from_backend(fake_backend)
sim_backend = AerSimulator(
    noise_model=noise_model,
    basis_gates=fake_backend.basis_gates,
    coupling_map=fake_backend.coupling_map
)

qc = QuantumCircuit(2)
qc.h(0); qc.cx(0,1)

pm = generate_preset_pass_manager(optimization_level=1, target=fake_backend.target)
cir_isa = pm.run(qc)

# Pad observable to match ISA qubit count (likely 5 on Manila)
nq = cir_isa.num_qubits
obs_label = "Z" + "I"*(nq-1)
obs_isa = SparsePauliOp.from_list([(obs_label, 1.0)])

est = EstimatorV2(mode=sim_backend)   # Runtime EstimatorV2 in local mode
# (No twirling here unless you enable it; Aer-only EstimatorV2 lacks twirling options)

job = est.run([(cir_isa, obs_isa)], precision=0.5)
res = job.result()
# print("⟨", obs_label, "⟩ =", res[0].value)


In [21]:
res

PrimitiveResult([PubResult(data=DataBin(evs=np.ndarray(<shape=(), dtype=float64>), stds=np.float64(0.0)), metadata={'target_precision': 0.5, 'shots': 4, 'circuit_metadata': {}})], metadata={'version': 2})

With twirling enabled, the distribution may show reduced bias from coherent errors, with error probabilities more evenly spread.



**5.1.6 Backend/simulator notes & quick validation**

Many options (DD, twirling) are hardware-only in effect; on ideal simulators, they have no observable impact.

For realistic simulation, use fake backends with noise models to mimic hardware behavior.

Validate settings by printing sampler.options before running or checking job metadata after.



In [1]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel
# The correct primitive import for Aer simulations
from qiskit_aer.primitives import SamplerV2 as Sampler
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
# Use a standard fake backend for demonstration
from qiskit_ibm_runtime.fake_provider import FakeManilaV2


# --- SETUP: Define Backend and Circuit ---
# Initialize the fake backend (required for noise model)
fake_backend = FakeManilaV2()

# Define a simple 2-qubit circuit (Bell state)
qc = QuantumCircuit(2, 2)
qc.h(0)
qc.cx(0, 1)
qc.measure([0, 1], [0, 1])

print(f"Original Circuit Qubits: {qc.num_qubits}")


# --- PART 1: Define the noisy Aer Simulator ---
# We keep the configuration of the custom simulator.
noise_model = NoiseModel.from_backend(fake_backend)
sim_backend = AerSimulator(
    noise_model=noise_model,
    basis_gates=fake_backend.basis_gates,
    coupling_map=fake_backend.coupling_map
)

# --- CRITICAL FIX ---
# Instantiate Sampler *without* the 'backend' argument, as suggested by the previous error.
sampler_fake = Sampler()


# --- PART 2: Transpile the circuit ---
# Transpile the circuit specifically for the fake backend's architecture
pm_fake = generate_preset_pass_manager(optimization_level=1, backend=fake_backend)
isa_qc_fake = pm_fake.run(qc)

print(f"Transpiled Circuit Qubits: {isa_qc_fake.num_qubits}")


# --- PART 3: Set options and Run ---
# Set shots option. For Aer Sampler, 'shots' is the standard key.
sampler_fake.options.shots = 2048

# Run the job with the transpiled circuit
job_fake = sampler_fake.run([isa_qc_fake])

# FIX: Use .quasi_dists[0] to get the quasi-probability distribution
# from the SamplerResult, which is the V1 primitive's expected output.
result = job_fake.result()
# dist_fake = result.quasi_dists[0]
dist_fake = result[0].data

print("\n--- Simulation Result ---")
# The result here is a QuasiDistribution (a dict-like object where values are floats)
print(f"Result Quasi-Probabilities (Noisy): {dist_fake}")

Original Circuit Qubits: 2
Transpiled Circuit Qubits: 5

--- Simulation Result ---
Result Quasi-Probabilities (Noisy): DataBin(c=BitArray(<shape=(), num_shots=1024, num_bits=2>))


For real hardware, use job.status() to monitor and job.result() to retrieve.



In [2]:
result

PrimitiveResult([SamplerPubResult(data=DataBin(c=BitArray(<shape=(), num_shots=1024, num_bits=2>)), metadata={'shots': 1024, 'circuit_metadata': {}, 'simulator_metadata': {'time_taken_parameter_binding': 5.667e-06, 'max_memory_mb': 18432, 'time_taken_execute': 0.000505875, 'omp_enabled': True, 'max_gpu_memory_mb': 0, 'parallel_experiments': 1}})], metadata={'version': 2})

**5.1.7 Sample Multiple Choice Questions**

**(5.1.1) Q1.** You construct a Bell state and sample with `SamplerV2` using 4000 default shots. Which distribution is most plausible on hardware?  
A. Equal probabilities for `00,01,10,11`  
B. Peaks at `00` and `11`, small counts at `01`/`10`  
C. Only `00` occurs  
D. Only `11` occurs  
<details><summary>Answer & rationales</summary>

- **Correct: B.** Ideal Bell ⇒ `00`/`11`; noise populates `01`/`10` slightly.  
- A: Equal over all 4 would imply a different circuit/state.  
- C/D: Single outcome would require measurement or state-prep that collapses fully.
</details>

**(5.1.1) Q2.** Where do you set default values that apply to subsequent sampler runs?  
A. In each circuit object  
B. In `sampler.options`  
C. In `QuantumCircuit.metadata`  
D. In the `run()` call’s `metadata`  
<details><summary>Answer & rationales</summary>

- **Correct: B.** Defaults live on `sampler.options`.  
- A/C/D: Do not set sampler execution defaults.
</details>

**(5.1.2) Q3.** You set `sampler.options.default_shots = 2048`. What does this affect?  
A. The number of qubits measured  
B. The number of shots per circuit unless overridden  
C. The number of DD pulses inserted  
D. The basis in which measurements are performed  
<details><summary>Answer & rationales</summary>

- **Correct: B.** Controls total repetitions.  
- A: Qubit count is circuit-defined.  
- C: DD pulses are controlled by DD options, not `default_shots`.  
- D: Basis is controlled by measurement/circuit, not shot count.
</details>

**(5.1.2) Q4.** Best lightweight way to confirm your default applies?  
A. Print `sampler.options` before running  
B. Reboot the kernel  
C. Check transpiler logs  
D. Increase optimization level  
<details><summary>Answer & rationales</summary>

- **Correct: A.** Echo options.  
- B/C/D: Not related to sampler defaults visibility.
</details>

**(5.1.3) Q5.** If you raise `default_shots` from 200 to 20000, what happens to variance in estimated probabilities?  
A. Increases  
B. Decreases  
C. Unchanged  
D. Randomly flips bitstrings  
<details><summary>Answer & rationales</summary>

- **Correct: B.** More shots ⇒ lower statistical variance.  
- A/C/D: Do not reflect shot-based sampling behavior.
</details>

**(5.1.3) Q6.** Which description matches shot-based sampler output?  
A. A single fidelity number  
B. Mapping from bitstrings to counts/probabilities  
C. A device temperature log  
D. OpenQASM only  
<details><summary>Answer & rationales</summary>

- **Correct: B.** Sampler returns distributions over outcomes.  
- A/C/D: Not the sampler’s output format.
</details>

**(5.1.4) Q7.** Enabling DD (XY4) primarily:  
A. Changes measurement basis  
B. Mitigates idle errors using calibrated pulse sequences  
C. Optimizes transpilation time  
D. Reduces shot usage  
<details><summary>Answer & rationales</summary>

- **Correct: B.** DD suppresses decoherence during idle periods.  
- A/C/D: Not the purpose of DD.
</details>

**(5.1.4) Q8.** On an ideal simulator, enabling DD typically:  
A. Dramatically changes outcome histograms  
B. Has no observable effect  
C. Forces an error  
D. Doubles shots automatically  
<details><summary>Answer & rationales</summary>

- **Correct: B.** No physical noise ⇒ no DD effect.  
- A/C/D: Incorrect.
</details>

**(5.1.5) Q9.** Pauli twirling is mainly used to:  
A. Randomize coherent errors into stochastic ones  
B. Eliminate all noise  
C. Guarantee zero error on hardware  
D. Speed up compilation  
<details><summary>Answer & rationales</summary>

- **Correct: A.** Twirling averages out coherent error directions.  
- B/C/D: Overstatements or unrelated.
</details>

**(5.1.5) Q10.** DD and twirling can be:  
A. Never combined  
B. Combined, depending on backend support  
C. Combined only on simulators  
D. Combined only with zero shots  
<details><summary>Answer & rationales</summary>

- **Correct: B.** They are compatible depending on backend.  
- A/C/D: Incorrect constraints.
</details>

**(5.1.6) Q11.** Which is most accurate?  
A. All options affect sim and hardware equally  
B. Many options are **hardware‑only** in effect  
C. Options are ignored on hardware  
D. Options are cosmetic only  
<details><summary>Answer & rationales</summary>

- **Correct: B.** DD/twirling effects are hardware/noise dependent.  
- A/C/D: Incorrect.
</details>

**(5.1.6) Q12.** When simulating to mimic hardware behavior, best practice is to:  
A. Use seed 0  
B. Mirror the device’s coupling map and noise model  
C. Disable measurements  
D. Run 1 shot  
<details><summary>Answer & rationales</summary>

- **Correct: B.** Device mirroring yields realistic behavior.  
- A/C/D: Not sufficient or relevant.
</details>